# Environment setup and imports

In [ ]:
!pip install evaluate #install eval library

In [ ]:
#importing libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
#hugging face
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          Trainer,TrainingArguments,pipeline)
from datasets import Dataset
import evaluate #for f1 and accuracy
#evaluation metrics
from sklearn.metrics import (accuracy_score,classification_report,confusion_matrix)